# Tanzu Build Service (TBS) - Level 100 - Installation & Demo

# Agenda:

 - <a href=#install>Installation of TBS</a>
 - <a href=#validate>Validate and Create a Project in K8s cluster using TBS</a>
 - <a href=#configure>Configure and Test Application using TBS</a>



## <a name='install'/>Installation of Tanzu Build Service

These are two setups to get a kickstart. With the automated version, just fill out the configuration of your environment and run the command. The manual step is more towards following the Official documentation.

- <a href=#auto>Automated Setup</a>
- <a href=#manual>Manual Setup</a>

### <a name='auto'/>Automated - Using pbsetup on GKE

Use the following steps to setup Tanzu Build Service: https://github.com/mcnichol/build-service-lab

### <a name='manual'/>Manual Setup

In this setup, we will be using TKGI environment running on GCP provisioned by [Tanzu Toolsmith](https://environments.toolsmiths.cf-app.com/).

- <a href=#validate-pks>Validate PKS Environment</a>
- <a href=#tbs-pvc>Create PVC for Build Service</a>
- <a href=#tbs-bits>Prerequisites & Setup for Tanzu Build Service</a>
- <a href=#tbs-install>Install Tanzu Build Service</a>


#### <a name='validate-pks'/>Validate K8s Environment

Login to your PKS environment:

In [ ]:
!pks login -u admin -p mEPoeWXsXC75IIP64dvbQUO4CEL-Xytr -a api.pks.ferndale.cf-app.com -k

In [ ]:
!pks plans

In [ ]:
!pks clusters

In [ ]:
!pks create-cluster tbs-cluster --external-hostname tbs-cluster.ferndale.cf-app.com --plan small

In [ ]:
!pks get-credentials ferndale-pks-cluster-1

In [ ]:
!pks cluster ferndale-pks-cluster-1

In [ ]:
!kubectl get po -A

#### <a name='tbs-pvc'/>Create Persistent Volume and Claim for TBS

Persistent Volume:

apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: fast
  annotations:
    storageclass.kubernetes.io/is-default-class: "true"
provisioner: kubernetes.io/vsphere-volume
parameters:
  datastore: datastore1
  diskformat: thin
  fstype: ext3


In [ ]:
!pwd

In [ ]:
!kubectl get StorageClasses

In [ ]:
!kubectl apply -f tmp/tbs-pcv.yaml

In [ ]:
!kubectl get pv

In [ ]:
!kubectl get pvc

#### <a name='tbs-bits'/>Prerequisites & Setup for Tanzu Build Service


##### Login to https://network.pivotal.io and download the following:

    - Tanzu Build Service: build-service-0.1.0.tgz
    - Duffle
    - PB CLI
    - Login to Docker Registry
    
    Steps to install CLIs

    - rename duffle-0.0.4-darwin to duffle
    - rename pb-0.1.0-darwin to pb
    - chmod +x duffle
    - chmod +x pb
    - mv duffle /usr/local/bin
    - mv pb /usr/local/bin

In [ ]:
!duffle version

In [ ]:
!pb version

In [ ]:
ls ../

##### Build Tanzu Build Service - credentials files

The credential.yml file is already built and for index.docker.io, we don't need to add the CA Cert. Also in the repo, we have added another credentials.yml as an example for harbor.

In [ ]:
cat ../tmp/build-service-credentials.yml

##### Login to Docker Registry and push Tanzu Build Service Bundle to Docker Hub

For this example we are using index.docker.io

In [ ]:
!docker login docker.io

Push the TBS Bundle to docker hub using the below command:

duffle relocate -f build-service-0.1.0.tgz -m relocated.json -p index.docker.io/<username>

Give the **path** where you have downloaded build-service-0.1.0.tgz or run it from that location

In [ ]:
!duffle relocate -f ./tmp/build-service-0.1.0.tgz -m ./tmp/relocated.json --repository-prefix=sameerhashmi

Once you run the above command, **relocated.json** will be created and a builder package will be created inside your DockerHub Repository.

![HARBOR-Buildservice](img/dockerhub-snapshot.png)

#### <a name='tbs-install'/>Install Build Service

Create a credentials.yml file with your kube confiog information and CA certs for registry

In [ ]:
cat ../tmp/build-service-credentials.yml

#### As above in the directory above where you have the following:

- build-service-credentials.yml
- buildserver tar file (build-service.0.1.0.tgz)
- relocated.json (generated in the above steps)

Fill out the remaining as per your environment and run the duffle command:


duffle install pbs-demo -c ./tmp/build-service-credentials.yml  \
    --set kubernetes_env=<k8s-cluster-name> \
    --set docker_registry=<DOCKER_REGISTRY> \
    --set docker_repository=<DOCKER_REPO> \
    --set registry_username=<REGISTYRY_USER_NAME> \
    --set registry_password=<REGISTYRY_PASSWORD> \
    --set custom_builder_image=default-builder \
    -f ./tmp/build-service-0.1.0.tgz \
    -m ./tmp/relocated.json


Avoiding duffle error: claim already exists
If an install fails or if you prefer to reset your Kubernetes cluster rather than uninstalling, you can remove existing duffle claims by deleting the files in ~/.duffle/claims. This will allow you to re-install without encountering the “claim already exists” error.



In [ ]:
!duffle install pbs-demo -c ./tmp/build-service-credentials.yml  \
    --set kubernetes_env=ferndale-pks-cluster-1 \
    --set docker_registry=index.docker.io \
    --set docker_repository=index.docker.io/sameerhashmi \
    --set registry_username="sameerhashmi" \
    --set registry_password="aa59054b-a169-4398-93c8-b2219a4ef734" \
    --set custom_builder_image=sameerhashmi/default-builder \
    -f ./tmp/build-service-0.1.0.tgz \
    -m ./tmp/relocated.json

If the duffle claim exisits, you might have to remove it and run the above command again.

In [ ]:
rm ~/.duffle/claims/pbs-demo.json

In [ ]:
!kubectl get ns

## <a name='validate'/> Validate and Create a Project in K8s cluster using TBS</a>

- Validate TBS Build
- Create Project in TBS


In [ ]:
!pb version

In [ ]:
!pb stack status

In [ ]:
!pb store list

### Create a project in TBS

Creating a project will also create a namespace in your Kubernetes cluster where your local image will run. 

In [ ]:
!pb project list

In [ ]:
!pb project create node-todo

In [ ]:
!kubectl get ns

Target the Project:

By targeting, this means that all subsequent pb commands will be against this project, including configuring secrets and adding users. 


In [ ]:
!pb project target node-todo

## <a name='configure'/>Configure and Test Application using TBS</a>

For demo purpose, we will first fork a spring-petclinic repository so that we can commit and push changes to github. Once configured with Tanzu Build Service, it will start building as soon any changes are made to the repo and push it to your configured docker registry

- Fork the repo https://github.com/migmartri/node-todo and clone it.
- Configure Secrets
- Configure Image on TBS
- Validate and Test


In [ ]:
cd ../../

In [ ]:
ls

In [ ]:
!git clone https://github.com/sameerhashmi/node-todo.git

In [ ]:
cd node-todo

### Configure Secrets

TBS will communicate to your version control system and your image registry. In this step we will create configurations and pass these credentials to Tanzu Build Service using the pb secrets command.

 - Github Repo
 - Docker Hub 
 
 These will apply the secrets to the project targeted. In our case, spring-petclinic

In [ ]:
mkdir tbs-config

Create two Files:

1 - Docker Hub Config name "dockerhub-config.yml" and fill in the details:

project: node-todo
registry: https://index.docker.io/v1/
username: <username>
password: <password-or-token>
    
    
2 - Github Config name "github-config.yml" and fill in the details:    

project: node-todo
repository: github.com/sameerhashmi/node-todo
username: <username>
password: <password-or-token>
    

In [ ]:
cat ./tbs-config/dockerhub-config.yml

In [ ]:
cat ./tbs-config/github-config.yml

In [ ]:
!pb secrets

In [ ]:
!pb secrets git apply -f ./tbs-config/github-config.yml

In [ ]:
!pb secrets registry apply -f ./tbs-config/dockerhub-config.yml

### Configure an Image


Final step is to configure Tanzu Build Service about what code we want to monitor and build. 

We will add another configuration for TBS to know which App it needs to build.

Create a configuration named spring-petclinic-config.yml and fill in the details:

source:
  git:
    url: <github-url>
    revision: master
image:
  tag: <docker-registry-url>


In [ ]:
cat ./tbs-config/node-todo-config.yml

Once the image configuration has been applied, Tanzu Build Service will scan the repo and start building your image. You can see this happening by running the pb command below to list all of the builds associated with an image tag. 

In [ ]:
!pb image apply -f tbs-config/node-todo-config.yml

In [ ]:
!pb image builds index.docker.io/sameerhashmi/node-todo

Verify the image in index.docker.io/sameerhashmi

In [ ]:
!pb image status index.docker.io/sameerhashmi/node-todo

In [ ]:
!pb image list 

In [ ]:
!pb project list

In [ ]:
!pb project target spring-music

In [ ]:
!pb image list

In [ ]:
!pb image builds index.docker.io/sameerhashmi/spring-music:latest